In [85]:
import pandas as pd
from datetime import date, timedelta


In [86]:
flow_df_banff = pd.read_csv('bow-river-level-at-banff.csv',parse_dates=[['Date', 'Time']])
flow_df_calgary = pd.read_csv('bow-river-flow-target-feature.csv', parse_dates=[['Date', 'Time']])
elbow_precip = pd.read_csv('elbow-ranger-station-precip.csv', parse_dates=[['Date', 'Time']])
elbow_bragg_creek = pd.read_csv('elbow-river-at-bragg-creek.csv', parse_dates=[['Date', 'Time']])
elbow_glenmore_dam = pd.read_csv('elbow-river-flow-below-glenmore-dam-target-feature.csv', parse_dates=[['Date', 'Time']])
snow_pillow_elbow = pd.read_csv('snow-pillow-elbow.csv', parse_dates=[['Date', 'Time']])
lake_louise_precip = pd.read_csv('precip-at-lake-louise.csv', parse_dates=[['Date', 'Time']])
lake_louise_snow_pillow = pd.read_csv('snow-pillow-lake-louise.csv', parse_dates=[['Date', 'Time']])

In [87]:
def format_data(df):
    df= df.resample('60min', on='Date_Time').mean()
    df = df[df.index.month > 3]
    df = df[df.index.month < 10]
    df = df[df.index.year >1999]
    df = df[df.index.year <=2020]
    df_parsed = df.fillna(method='backfill')
    return df_parsed

In [88]:
flow_df_calgary_parsed = format_data(flow_df_calgary)
flow_df_banff_parsed = format_data(flow_df_banff)
elbow_precip_parsed = format_data(elbow_precip)
elbow_bragg_creek_parsed = format_data(elbow_bragg_creek)
elbow_glenmore_dam_parsed = format_data(elbow_glenmore_dam)
snow_pillow_elbow_parsed = format_data(snow_pillow_elbow)
lake_louise_precip_parsed = format_data(lake_louise_precip)
lake_louise_snow_pillow_parsed = format_data(lake_louise_snow_pillow)

In [89]:
flow_df_calgary_parsed = flow_df_calgary_parsed.rename(columns={'Value(m3/s)':'calgary_bow_flow(m3/s)'})
flow_df_banff_parsed = flow_df_banff_parsed.rename(columns={'Value(m3/s)':'banff_bow_flow(m3/s)'})
elbow_precip_parsed = elbow_precip_parsed.rename(columns={'Value(mm)': 'elbow_precip(mm)'})

In [90]:
df = pd.merge(flow_df_calgary_parsed, flow_df_banff_parsed, left_index=True, right_index=True)
df = pd.merge(df, elbow_precip_parsed, left_index=True, right_index=True)
df = pd.merge(df, elbow_bragg_creek_parsed, left_index=True, right_index=True)
df = pd.merge(df, elbow_glenmore_dam_parsed, left_index=True, right_index=True)
df = pd.merge(df, snow_pillow_elbow_parsed, left_index=True, right_index=True)
df = pd.merge(df,lake_louise_precip_parsed, left_index=True, right_index=True)
df = pd.merge(df,lake_louise_snow_pillow_parsed, left_index=True, right_index=True)

In [91]:
dataframe2 = df

In [92]:
dataframe2

,calgary_bow_flow(m3/s),banff_bow_flow(m3/s),elbow_precip(mm),elbow-river-bragg-creek(m3/s),elbow-river-glenmore-dam(m3/s),snow-pillow-elbow(mm),lake-louise-precip(mm),lake-louise-snow-pillow(mm)
Date_Time,,,,,,,,
2000-04-01 00:00:00,75.276250,8.217030,0.0,0.000000,4.871450,380.39999,0.0,355.23561
2000-04-01 01:00:00,75.126810,8.217150,0.0,0.000000,5.881160,381.35001,0.0,355.23561
2000-04-01 02:00:00,74.977350,8.205775,0.0,0.000000,5.985200,381.35001,0.0,355.23561
2000-04-01 03:00:00,74.677900,8.240400,0.0,0.000000,5.985250,381.35001,0.0,355.23561
2000-04-01 04:00:00,72.278450,8.275020,0.0,0.000000,5.777300,382.29999,0.0,355.23561
...,...,...,...,...,...,...,...,...
2020-09-30 19:00:00,59.615721,23.038813,0.0,5.199962,2.469573,1.60889,0.0,0.00000
2020-09-30 20:00:00,58.268692,22.992368,0.0,5.202278,2.469517,1.61096,0.0,0.00000
2020-09-30 21:00:00,57.964387,22.993643,0.0,5.192512,2.469463,1.61303,0.0,0.00000


In [41]:
flow_df_calgary_parsed.to_pickle('../parsed/dataframe1.pkl')

In [100]:
dataframe2.to_pickle('../parsed/dataframe2.pkl')


In [ ]:
dataframe2.to_csv('../parsed/dataframe2.csv',index=True, index_label='Date_Time')

In [115]:
df4 = pd.read_csv('../parsed/dataframe2.csv')

In [116]:
df4

,calgary_bow_flow(m3/s),banff_bow_flow(m3/s),elbow_precip(mm),elbow-river-bragg-creek(m3/s),elbow-river-glenmore-dam(m3/s),snow-pillow-elbow(mm),lake-louise-precip(mm),lake-louise-snow-pillow(mm)
0,75.276250,8.217030,0.0,0.000000,4.871450,380.39999,0.0,355.23561
1,75.126810,8.217150,0.0,0.000000,5.881160,381.35001,0.0,355.23561
2,74.977350,8.205775,0.0,0.000000,5.985200,381.35001,0.0,355.23561
3,74.677900,8.240400,0.0,0.000000,5.985250,381.35001,0.0,355.23561
4,72.278450,8.275020,0.0,0.000000,5.777300,382.29999,0.0,355.23561
...,...,...,...,...,...,...,...,...
92227,59.615721,23.038812,0.0,5.199962,2.469573,1.60889,0.0,0.00000
92228,58.268692,22.992368,0.0,5.202278,2.469517,1.61096,0.0,0.00000
92229,57.964387,22.993643,0.0,5.192512,2.469463,1.61303,0.0,0.00000
92230,57.660082,22.963793,0.0,5.192412,2.469408,1.61509,0.0,0.00000
